<a href="https://colab.research.google.com/github/HenningBuhl/DLML/blob/master/GAN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DLML - GAN

Sources:

1. ...
1. ...

## TODO

In [0]:
"""

- Implement GAN without Convolutions

- Change single latent variable and inspect ramifications in generated image.
- Grid of latent space and sliders to explore it

- Use different data: Celeb Faces, Knitting patterns, Pokemon, Make own data, ...

- t-SNE of latent space
- 2D Manifold with classifier labels

DIFFICULT! (better create new notebooks to keep code clean and retain old versions of GAN)
- Use different loss function?
- Use Pro GAN
- Use VAE GAN
- Use Style GAN
- Use Cycle GAN
- Use Conditional GAN

"""

## Sources

- https://github.com/eriklindernoren/Keras-GAN
- https://machinelearningmastery.com/how-to-code-generative-adversarial-network-hacks/
- https://www.youtube.com/watch?v=dCKbRCUyop8
- https://github.com/google/compare_gan
- https://arxiv.org/pdf/1806.11382.pdf
- https://arxiv.org/pdf/1812.04948.pdf
- https://arxiv.org/pdf/1710.10196.pdf
- https://arxiv.org/pdf/1801.04406.pdf
- https://arxiv.org/pdf/1512.09300.pdf
- https://medium.com/@jonathan_hui/gan-gan-series-2d279f906e7b
- https://gist.github.com/korakot/8409b3feec20f159d8a50b0a811d3bca
- https://www.is.mpg.de/uploads_file/attachment/attachment/426/GAN_convergence.pdf
- https://arxiv.org/pdf/1706.04156.pdf
- 


## Random Seeds

In [0]:
# Set Numpy seed.
from numpy.random import seed
seed(1)

# Set TensorFlow seed.
from tensorflow import set_random_seed
set_random_seed(2)

## Imports

In [0]:
import keras
import keras.backend as K

from keras.layers import Input, Dense, Flatten, Reshape, Conv2D, Conv2DTranspose, MaxPooling2D, UpSampling2D
from keras.layers import BatchNormalization, Dropout, ZeroPadding2D
from keras.layers import Activation, LeakyReLU, ReLU

from keras.models import Sequential, Model, load_model
from keras.initializers import RandomNormal, glorot_normal
from keras.optimizers import Adam
from keras.utils import to_categorical
from keras.datasets import mnist, fashion_mnist, cifar10

import os
import time
import datetime
import numpy as np
import pandas as pd
from IPython.display import HTML

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.animation as animation
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)

## Functions

In [0]:
# Return datetime.
def get_datetime(seconds):
    return str(datetime.timedelta(seconds=seconds))

In [0]:
# Return moving average of argument.
def ema(data, window_size=100):
    cumsum_vec = np.cumsum(np.insert(data, 0, 0)) 
    ma_vec = (cumsum_vec[window_size:] - cumsum_vec[:-window_size]) / window_size
    return ma_vec

In [0]:
# Return the norm of all gradients of trinable parameters (only works in training).
def get_gradient_norm(model):
    with K.name_scope('grad_norm'):
        grads = K.gradients(model.total_loss, model.trainable_weights)
        norm = K.sqrt(sum([K.sum(K.square(g)) for g in grads]))
    return norm

# Data

In [0]:
# Data dimensions.
x_dim = 28
y_dim = 28
channels = 1
img_shape = (x_dim, y_dim, channels)

In [0]:
# Load data.
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
# Concatonate train and test data.
#x_train = np.concatenate((x_train, x_test))

# Add channel dimension.
x_train = x_train.reshape(-1, x_dim, y_dim, channels)
x_test = x_test.reshape(-1, x_dim, y_dim, channels)

# Normalize data to interval (-1, 1).
x_train = x_train / 255 * 2 - 1
x_test = x_test / 255 * 2 - 1

In [0]:
# Print data shape.
print(x_train.shape)

In [0]:
plt.imshow(x_train[6].reshape(x_dim, y_dim), cmap='gray_r')
plt.show()

# GAN

## Hyper Parameters

In [0]:
iterations = 10000
batch_size = 128
latent_dim = 100

init = glorot_normal() #RandomNormal(mean=0, stddev=0.02)
optimizer = Adam(lr=0.0002, beta_1=0.5)

## Generator

In [0]:
# Build generator model.
z = Input(shape=(latent_dim,))
x = Dense(128 * 7 * 7, kernel_initializer=init)(z)
x = LeakyReLU(0.2)(x)
x = Reshape((7, 7, 128))(x)

x = Conv2D(128, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(128, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2D(64, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2DTranspose(64, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)

x = Conv2D(channels, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = Activation("tanh")(x)

generator = Model(z, x)
generator.name = "generator"
generator.summary()

## Discriminator

In [0]:
# Build discriminator model.
img_input = Input(shape=img_shape)
x = Conv2D(32, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(img_input)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(64, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(128, kernel_size=3, strides=(2, 2), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Conv2D(256, kernel_size=3, strides=(1, 1), padding="same", kernel_initializer=init)(x)
x = BatchNormalization()(x)
x = LeakyReLU(0.2)(x)
x = Dropout(0.25)(x)

x = Flatten()(x)
x = Dense(1, kernel_initializer=init)(x)
x = Activation("sigmoid")(x)

discriminator = Model(img_input, x)
discriminator.name = "discriminator"
discriminator.compile(loss='binary_crossentropy',
    optimizer=optimizer,
    metrics=['accuracy'])

discriminator.metrics_names.append("grad_norm")
discriminator.metrics_tensors.append(get_gradient_norm(discriminator))

discriminator.summary()

## Combined Model

In [0]:
# For the combined model we will only train the generator.
discriminator.trainable = False

# Combined model (stacked generator and discriminator).
combined = Model(z, discriminator(generator(z)))
combined.name = "combined"
combined.compile(loss='binary_crossentropy', optimizer=optimizer)

combined.metrics_names.append("grad_norm")
combined.metrics_tensors.append(get_gradient_norm(combined))

combined.summary()

## Training

In [0]:
# Loss history.
losses = {'d_loss' : [],
          'd_acc' : [],
          'g_loss' : [],
          'd_grad_norm' : [],
          'g_grad_norm' : []}

In [0]:
# Savely create path.
save_path_image = "images/"
if not os.path.exists(save_path_image):
    os.mkdir(save_path_image)

In [0]:
# Adversarial ground truths.
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

# Start time measurement.
start = time.time()

# Save interval settings.
save_interval = 50
use_static_noise = True
r, c = 5, 5
static_noise = np.random.normal(0, 1, (r * c, latent_dim))
save_paths = []

for iteration in range(iterations):

    # ---------------------
    #  Train Discriminator
    # ---------------------

    # Select a random half of images.
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train[idx]

    # Sample noise and generate a batch of new images.
    noise = np.random.normal(0, 1, (batch_size, latent_dim))
    gen_imgs = generator.predict(noise)

    # Train the discriminator (real classified as ones and generated as zeros).
    d_loss_real, d_acc_real, d_grad_norm_real = discriminator.train_on_batch(imgs, valid)
    d_loss_fake, d_acc_fake, d_grad_norm_fake = discriminator.train_on_batch(gen_imgs, fake)

    # Calculate discriminator metrics (average of real and fake batches) to record.
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)
    d_acc = 0.5 * np.add(d_acc_real, d_acc_fake)
    d_grad_norm = 0.5 * np.add(d_grad_norm_real, d_grad_norm_fake)

    # Log discriminator metrics.
    losses['d_loss'].append(d_loss.tolist())
    losses['d_acc'].append(d_acc.tolist())
    losses['d_grad_norm'].append(d_grad_norm.tolist())

    # ---------------------
    #  Train Generator
    # ---------------------

    # Sample novel noise.
    noise = np.random.normal(0, 1, (batch_size, latent_dim))

    # Train the generator (wants discriminator to mistake images as real).
    g_loss, g_grad_norm = combined.train_on_batch(noise, valid)
    
    # Log generator metrics.
    losses['g_loss'].append(g_loss)
    losses['g_grad_norm'].append(g_grad_norm)

    # Print progress.
    print("Iteration: {:5d} [D loss: {:1.5f}, acc.: {:5.2f}%, grad norm: {:5.2f}] [G loss: {:1.5f}, grad norm: {:5.2f}]".format(
        iteration + 1, d_loss, 100 * d_acc, d_grad_norm, g_loss, g_grad_norm))

    # Save generated image samples.
    if (iteration + 1) % save_interval == 0 or (iteration + 1) == 1:
        if use_static_noise:
            noise = static_noise # Use static noise.
        else:
            noise = np.random.normal(0, 1, (r * c, latent_dim)) # Use new radom noise every save iteration.
        gen_imgs = generator.predict(noise)

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
                axs[i,j].axis('off')
                cnt += 1
        fig.suptitle("Iteration: {:d}".format(iteration + 1))
        iter_save_path = "{:s}iteration_{:d}.png".format(save_path_image, iteration + 1)
        save_paths.append(iter_save_path)
        fig.savefig(iter_save_path)
        plt.show()

    # Finish time measurement.
    if (iteration + 1) == iterations:
        end = time.time()
        training_time = end - start
        print('Finished training in {:s}'.format(get_datetime(training_time)))

In [0]:
# Savely create path.
save_path_training = "training/"
if not os.path.exists(save_path_training):
    os.mkdir(save_path_training)

In [0]:
# Create animation of generated images during trainig.
fig = plt.figure()
ims = []
for save_path in save_paths:
    im = plt.imshow(mpimg.imread(save_path) , cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('{:s}training.mp4'.format(save_path_training))
HTML(ani.to_html5_video())

In [0]:
# Plot generator and discriminator loss.
d_loss = np.array(losses['d_loss'])
g_loss = np.array(losses['g_loss'])

plt.plot(d_loss, label="D loss")
plt.plot(g_loss, label="G loss")

plt.legend(loc="best")
plt.savefig("{:s}D and G loss".format(save_path_training))
plt.show()

In [0]:
# Plot discriminator accuracy.
d_acc = np.array(losses['d_acc'])

d_acc_ema = ema(d_acc, window_size=100)

plt.plot(d_acc, label="D acc")
plt.plot(d_acc_ema, label="D acc (ema)")

plt.legend(loc="best")
plt.savefig("{:s}D acc".format(save_path_training))
plt.show()

In [0]:
# Plot generator and discriminator gradient norm.
d_grad_norm = np.array(losses['d_grad_norm'])
g_grad_norm = np.array(losses['g_grad_norm'])

plt.plot(d_grad_norm, label="D grad norm")
plt.plot(g_grad_norm, label="G grad norm")

plt.legend(loc="best")
plt.savefig("{:s}D and G grad norm".format(save_path_training))
plt.show()

## Save and Load GAN Models

In [0]:
# Savely create path.
model_path = "models/"
if not os.path.exists(model_path):
    os.mkdir(model_path)

discriminator_path = model_path + "discriminator.h5"
generator_path = model_path + "generator.h5"
combined_path = model_path + "combined.h5"

In [0]:
# Save models.
discriminator.save(discriminator_path)
generator.save(generator_path)
combined.save(combined_path)

In [0]:
# Load models.
generator = load_model(generator_path)
discriminator = load_model(discriminator_path)
combined = load_model(combined_path)

In [0]:
discriminator.summary()

In [0]:
generator.summary()

In [0]:
combined.summary()

## Generate from Latent Space

In [0]:
# Savely create path.
save_path_gen_image = "gen_images/"
if not os.path.exists(save_path_gen_image):
    os.mkdir(save_path_gen_image)

In [0]:
# Generate images from random noise z.
r, c = 5, 5
noise = np.random.normal(0, 1, (r * c, latent_dim))
gen_imgs = generator.predict(noise)

fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
        axs[i,j].axis('off')
        cnt += 1

fig.suptitle("Generated Images")
fig.savefig("{:s}gen_image.png".format(save_path_gen_image))
plt.show()
plt.close()

In [0]:
# Visualize latent space travel from point A to point B.
intermediate_steps = 100

# Points in z.
point_a = np.random.normal(0, 1, (1, latent_dim))
point_b = np.random.normal(0, 1, (1, latent_dim))
#point_a = np.full((1, latent_dim), -1)
#point_b = np.full((1, latent_dim), 1)

points_travel = np.array(
    [point_a + (point_b - point_a) * x / intermediate_steps for x in range(0, 1 + intermediate_steps)]).reshape(-1, latent_dim)

# Images generated from z.
image_a = generator.predict(point_a)
image_b = generator.predict(point_b)
images_travel = generator.predict(points_travel)

# Show origin and destination image.
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image_a.reshape(x_dim, y_dim), cmap='gray_r')
axs[0].axis('off')
axs[0].title.set_text("Origin image")
axs[1].imshow(image_b.reshape(x_dim, y_dim), cmap='gray_r')
axs[1].axis('off')
axs[1].title.set_text("Destination image")
plt.show()

# Create animation.
fig = plt.figure()
ims = []
for image_travel in images_travel:
    im = plt.imshow(image_travel.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel.mp4')
HTML(ani.to_html5_video())

In [0]:
# Travel freely through latent space.
n_travel = 1000
points = []
images = []

# Initial points.
#point_prev = np.full((1, latent_dim), 0)
point_prev = np.random.normal(0, 1, (1, latent_dim))
point_curr = point_prev + 0.1 * np.random.uniform(-0.1, 0.1, (1, latent_dim))

# Travel.
print("Calculating travel points...")
for i in range(n_travel):
    direction = (point_prev - point_curr) + 0.25 * np.random.uniform(-0.1, 0.1, (1, latent_dim))
    point = point_prev + direction
    point = point.clip(min=-3, max=3)
    points.append(point)
    point_curr = point_prev
    point_prev = point

points = np.array(points).reshape(-1, latent_dim)
images = generator.predict(points)

# Create animation.
print("Creating animation...")
fig = plt.figure()
ims = []
for image in images:
    im = plt.imshow(image.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel_n.mp4')
HTML(ani.to_html5_video())

In [0]:
# Grid with sliders...






In [0]:
# 2D Manifold,
if latent_dim == 2:
    m = 1
    n = 30

    figure = np.zeros((x_dim * n, y_dim * n))
    #figure = plt.figure(figsize=(12,8))

    # Sample n points within std_devs.
    grid_x = np.linspace(-m, m, n)
    grid_y = np.linspace(-m, m, n)

    # Draw n by n grid.
    for i, yi in enumerate(grid_x):
        for j, xi in enumerate(grid_y):
            z_sample = np.array([[xi, yi]])
            x_decoded = generator.predict(z_sample)
            digit = x_decoded[0].reshape(x_dim, y_dim)
            figure[i * x_dim: (i + 1) * x_dim,
                  j * y_dim: (j + 1) * y_dim] = digit

    plt.figure(figsize=(24, 12))
    plt.imshow(figure)
    plt.savefig("2d_manifest_grid.png")
    plt.show()

# Classifier

## Training

In [0]:
# Create classifier model.
classifier = Sequential()
classifier.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=img_shape))
classifier.add(Conv2D(64, (3, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
classifier.add(Dropout(0.25))
classifier.add(Flatten())
classifier.add(Dense(128, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(10, activation='softmax'))

classifier.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [0]:
# Train classifier.
classifier_history = classifier.fit(x_train, y_train,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_split=0.1)

In [0]:
# Plot training and validation loss.
plt.plot(classifier_history.history['loss'], label="loss")
plt.plot(classifier_history.history['val_loss'], label="val loss")

plt.legend(loc="best")
plt.show()
plt.close()

In [0]:
# Plot training and validation accuracy.
plt.plot(classifier_history.history['acc'], label="accuracy")
plt.plot(classifier_history.history['val_acc'], label="val accuracy")

plt.legend(loc="best")
plt.show()
plt.close()

In [0]:
# Print testing loss and accuracy.
test_eval = classifier.evaluate(x_test, y_test)
print("Testing Loss:    {:1.7f}".format(test_eval[0]))
print("Testing Accracy: {:5.2f}%".format(test_eval[1] * 100))

## Save and Load Classifier Model

In [0]:
# Savely create path.
model_path = "models/"
if not os.path.exists(model_path):
    os.mkdir(model_path)

classifier_path = model_path + "classifier.h5"

In [0]:
# Save models.
classifier.save(classifier_path)

In [0]:
# Load models.
classifier = load_model(classifier_path)

## Classifying Generated Images

In [0]:
# Savely create path.
save_path_gen_classify = "gen_classify/"
if not os.path.exists(save_path_gen_classify):
    os.mkdir(save_path_gen_classify)

In [0]:
# Plot generated images with classifier labels.
r, c = 5, 5
noise = np.random.normal(0, 1, (r * c, latent_dim))
gen_imgs = generator.predict(noise)
preds = classifier.predict(gen_imgs)

fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
        axs[i,j].axis('off')
        axs[i,j].title.set_text("Pred:{:d}".format(np.argmax(preds[cnt])))
        cnt += 1
fig.suptitle("Generated Images Classified")
fig.savefig("{:s}gen_classify.png".format(save_path_gen_classify))
plt.show()
plt.close()

In [0]:
# Create 2D Manifold with classifier labels.
if latent_dim == 2:
    pass




## Classifying Hand-Drawn Images

In [0]:
from IPython.display import HTML, Image
from google.colab.output import eval_js
from base64 import b64decode
from PIL import Image
import cv2

canvas_html = """
<canvas width=%d height=%d style="border:1px solid #000000;"></canvas>
<button>Finish</button>
<script>
var canvas = document.querySelector('canvas')
var ctx = canvas.getContext('2d')
ctx.lineWidth = %d
var button = document.querySelector('button')
var mouse = {x: 0, y: 0}
canvas.addEventListener('mousemove', function(e) {
  mouse.x = e.pageX - this.offsetLeft
  mouse.y = e.pageY - this.offsetTop
})
canvas.onmousedown = ()=>{
  ctx.beginPath()
  ctx.moveTo(mouse.x, mouse.y)
  canvas.addEventListener('mousemove', onPaint)
}
canvas.onmouseup = ()=>{
  canvas.removeEventListener('mousemove', onPaint)
}
var onPaint = ()=>{
  ctx.lineTo(mouse.x, mouse.y)
  ctx.stroke()
}
var data = new Promise(resolve=>{
  button.onclick = ()=>{
    resolve(canvas.toDataURL('image/png'))
  }
})
</script>
"""

def draw(filename='drawing.png', w=200, h=200, line_width=1):
    display(HTML(canvas_html % (w, h, line_width)))
    data = eval_js("data")
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f:
        f.write(binary)
    return len(binary)

In [0]:
# Let user draw.
side_length = 300
draw(w=side_length, h=side_length, line_width=side_length // 15)

# Pre-process image.
drawing = Image.open('drawing.png')
drawing = np.array(drawing)
drawing = drawing[:, :, 3]
drawing = cv2.resize(drawing, dsize=(x_dim, y_dim), interpolation=cv2.INTER_AREA)
drawing = drawing / 255 * 2 - 1

# Make prediction.
pred = classifier.predict(drawing.reshape(1, x_dim, y_dim, channels))

# Show result.
plt.imshow(drawing, cmap='gray_r')
plt.title("Pred: {:d}".format(np.argmax(pred[0])))
plt.axis('off')
plt.show()
plt.close()

# Playground

# Novel Code

# Deprecated Code

In [0]:
# Travel through latent space (uneven path).
r, c = 5, 5
init_noise = np.random.normal(0, 1, (latent_dim,))
noise_center = np.tile(init_noise, (r * c, 1))
noise = noise_center + np.random.normal(0, 0.2, (r * c, latent_dim))
gen_imgs = generator.predict(noise)

fig, axs = plt.subplots(r, c)
cnt = 0
for i in range(r):
    for j in range(c):
        axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray_r')
        axs[i,j].axis('off')
        cnt += 1
fig.suptitle("Generated Images")
fig.savefig("{:s}gen_image_{:d}.png".format(save_path_gen_image, gen_count))
plt.show()
plt.close()

gen_count += 1

In [0]:
# Generate from constant value.
r, c = 5, 5
noise = np.full((1, latent_dim), -0.5)
gen_imgs = generator.predict(noise)

plt.imshow(gen_imgs[0, :, :, 0], cmap='gray_r')
plt.title("Generated Image")
plt.show()
plt.close()

In [0]:
# Explore latent space by taking random mini steps.
intermediate_steps = 1
n_travel = 50
images_travel = []
images = []

# Points in z.
#point_init = np.random.normal(0, 1, (1, latent_dim))
point_init = np.full((1, latent_dim), -5)
point_b = point_init
images.append(generator.predict(point_b))

for i in range(n_travel):
    point_a = point_b

    point_b = point_a + np.random.uniform(low=-0.1, high=0.5)
    
    points_travel = np.array(
        [point_a + (point_b - point_a) * x / intermediate_steps for x in range(0, 1 + intermediate_steps)]).reshape(-1, latent_dim)

    # Images generated from z.
    image_a = generator.predict(point_a)
    image_b = generator.predict(point_b)
    images.append(image_b)
    images_travel.append(points_travel)
images_travel = np.array(images_travel).reshape(-1, latent_dim)
images_travel = generator.predict(images_travel)

# Create animation.
fig = plt.figure()
ims = []
for image_travel in images_travel:
    im = plt.imshow(image_travel.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel_steps.mp4')
HTML(ani.to_html5_video())

In [0]:
# Visualize latent space travel through multiple points.
intermediate_steps = 25
n_travel = 50
images_travel = []
images = []

# Points in z.
point_init = np.random.normal(0, 1, (1, latent_dim))
point_b = point_init
images.append(generator.predict(point_b))

for i in range(n_travel):
    point_a = point_b

    if i == n_travel - 1:
        point_b = point_init
    else:
        point_b = np.random.normal(0, 1, (1, latent_dim))
    
    points_travel = np.array(
        [point_a + (point_b - point_a) * x / intermediate_steps for x in range(0, 1 + intermediate_steps)]).reshape(-1, latent_dim)

    # Images generated from z.
    image_a = generator.predict(point_a)
    image_b = generator.predict(point_b)
    images.append(image_b)
    images_travel.append(points_travel)
images_travel = np.array(images_travel).reshape(-1, latent_dim)
images_travel = generator.predict(images_travel)

# Show all traversed images.
fig, axs = plt.subplots(1, n_travel)
for (image, ax) in zip(images, axs):
    #plt.figure(figsize=(2,2))
    ax.imshow(image.reshape(x_dim, y_dim), cmap='gray_r')
    ax.axis('off')
    #plt.title()
plt.show()

# Create animation.
fig = plt.figure()
ims = []
for image_travel in images_travel:
    im = plt.imshow(image_travel.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel_n.mp4')
HTML(ani.to_html5_video())

In [0]:
# Perform arithmetics on latent vectors.

# Points in z.
point_a = np.random.normal(0, 1, (1, latent_dim))
point_b = np.random.normal(0, 1, (1, latent_dim))
#point_a = np.full((1, latent_dim), -1)
#point_b = np.full((1, latent_dim), 1)

# Images generated from z.
image_a = generator.predict(point_a)
image_b = generator.predict(point_b)

# Generate images after arithmetics operations
point_a_minus_b = point_a - point_b
point_a_plus_b = point_a + point_b
image_a_minus_b = generator.predict(point_a_minus_b)
image_a_plus_b = generator.predict(point_a_plus_b)

# Show image a and iamge b image.
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image_a.reshape(x_dim, y_dim), cmap='gray_r')
axs[0].axis('off')
axs[0].title.set_text("Image A")
axs[1].imshow(image_b.reshape(x_dim, y_dim), cmap='gray_r')
axs[1].axis('off')
axs[1].title.set_text("Image B")
plt.show()

# Show newly calculated images.
fig, axs = plt.subplots(1, 2)
axs[0].imshow(image_a_minus_b.reshape(x_dim, y_dim), cmap='gray_r')
axs[0].axis('off')
axs[0].title.set_text("A - B")
axs[1].imshow(image_a_plus_b.reshape(x_dim, y_dim), cmap='gray_r')
axs[1].axis('off')
axs[1].title.set_text("A + B")
plt.show()

"""
points_travel = np.array(
    [point_a + (point_b - point_a) * x / intermediate_steps for x in range(0, 1 + intermediate_steps)]).reshape(-1, latent_dim)

images_travel = generator.predict(points_travel)

# Create animation.
fig = plt.figure()
ims = []
for image_travel in images_travel:
    im = plt.imshow(image_travel.reshape(x_dim, y_dim), cmap='gray_r', animated=True)
    plt.axis('off')
    ims.append([im])
plt.close()

ani = animation.ArtistAnimation(fig, ims, interval=50)
ani.save('latent_travel.mp4')
HTML(ani.to_html5_video())
"""